Tune learning rate (-r) and regularization strength (-b).
Do randomized grid search.

In [1]:
import numpy as np
import os
import re
import subprocess

In [6]:
def parse_best_loss(run_result):
    lines = run_result.stdout.decode().split('\n')
    best_loss_line = [l for l in lines if 'best loss' in l]
    if best_loss_line:
        best_loss_line = best_loss_line[0]
        best_loss = re.search(r'best loss: (\d+.\d+)', best_loss_line).group(1)
        return float(best_loss)
    else:
        # No line matches 'best loss' if early stopping wasn't necessary.
        # Given the fact that FFM is very likely to overfit, this result
        # is most likely not of our interest.
        return np.nan

def eval_param(train_set, validation_set, model_type='2', learning_rate=0.2, reg_param=1e-5):
    xlearn_train = '~/code/xlearn/build/xlearn_train'
    command = [xlearn_train, train_set,
          '-s', str(model_type),
         '-v', validation_set,
          '-r', str(learning_rate),
               '-b', str(reg_param)]
    command = ' '.join(command)
    result = subprocess.run(command, stdout=subprocess.PIPE, shell=True)
    return parse_best_loss(result)

def log_uniform(low=0, high=1, size=None):
    return np.power(10, np.random.uniform(low, high, size))

def eval_param_ls(train_set, validation_set, learning_rates, reg_params):
    scores = [eval_param(train_set, validation_set, lr, reg)
              for lr, reg in zip(learning_rates, reg_params)]
    print(scores)
    maxidx = np.nanargmin(np.array(scores))
    max_score = scores[maxidx]
    best_lr = learning_rates[maxidx]
    best_reg = reg_params[maxidx]
    return max_score, best_lr, best_reg, scores

In [7]:
size=16
learning_rates = log_uniform(-2,0,size) # default = 0.2
reg_params = log_uniform(-6,4,size) # default = 1e-5
print(learning_rates)
print(reg_params)

[0.05535535 0.5857477  0.05374603 0.06783252 0.02074139 0.04612233
 0.32910007 0.14424511 0.01100821 0.41153041 0.13126246 0.06129916
 0.3736754  0.02371638 0.40141611 0.01787572]
[2.64424247e+01 5.92750326e-05 2.49799312e-02 4.79095577e-02
 7.43024019e+03 2.28875503e-05 2.22506996e+02 7.55037980e-04
 5.36140667e+01 5.20784311e-05 3.39181974e+02 3.76248982e-01
 2.57895235e-03 6.47502203e-04 8.12278280e+01 2.70658691e-05]


In [10]:
project_path = '~/code/avazu-ctr/'
xlearn_train = '~/code/xlearn/build/xlearn_train'
train_set = os.path.join(project_path, 'ffm-data/train_site_small.ffm')
validation_set = os.path.join(project_path, 'ffm-data/validate_site_small.ffm')

In [11]:
eval_param_ls(train_set, validation_set, learning_rates, reg_params)

[2.264441, nan, nan, 0.449428, 3.4028234663852886e+38, nan, 3.4028234663852886e+38, nan, 3.4028234663852886e+38, nan, 3.4028234663852886e+38, 0.455382, nan, nan, 3.4028234663852886e+38, nan]


(0.449428, 0.06783252480163421, 0.04790955765446936)